In [10]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

import argparse
import time
import torch.optim as optim
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

from utils.utils_ import log_string, plot_train_val_loss
from utils.utils_ import count_parameters, load_data

from model.model_ import GMAN
from model.train import train
from model.test import test


In [17]:
parser = argparse.ArgumentParser()
parser.add_argument('--time_slot', type=int, default=5,
                    help='a time step is 5 mins')
parser.add_argument('--num_his', type=int, default=12,
                    help='history steps')
parser.add_argument('--num_pred', type=int, default=12,
                    help='prediction steps')
parser.add_argument('--L', type=int, default=1,
                    help='number of STAtt Blocks')
parser.add_argument('--K', type=int, default=8,
                    help='number of attention heads')
parser.add_argument('--d', type=int, default=8,
                    help='dims of each head attention outputs')
parser.add_argument('--train_ratio', type=float, default=0.7,
                    help='training set [default : 0.7]')
parser.add_argument('--val_ratio', type=float, default=0.1,
                    help='validation set [default : 0.1]')
parser.add_argument('--test_ratio', type=float, default=0.2,
                    help='testing set [default : 0.2]')
parser.add_argument('--batch_size', type=int, default=32,
                    help='batch size')
parser.add_argument('--max_epoch', type=int, default=1,
                    help='epoch to run')
parser.add_argument('--patience', type=int, default=10,
                    help='patience for early stop')
parser.add_argument('--learning_rate', type=float, default=0.001,
                    help='initial learning rate')
parser.add_argument('--decay_epoch', type=int, default=10,
                    help='decay epoch')
parser.add_argument('--traffic_file', default='./data/pems-bay.h5',
                    help='traffic file')
parser.add_argument('--SE_file', default='./data/SE(PeMS).txt',
                    help='spatial embedding file')
parser.add_argument('--model_file', default='./data/GMAN.pkl',
                    help='save the model to disk')
parser.add_argument('--log_file', default='./data/log',
                    help='log file')
args, unknown = parser.parse_known_args()
log = open(args.log_file, 'w')
log_string(log, str(args)[10: -1])
T = 24 * 60 // args.time_slot  # Number of time steps in one day
# load data
log_string(log, 'loading data...')
(trainX, trainTE, trainY, valX, valTE, valY, testX, testTE,
testY, SE, mean, std) = load_data(args)
log_string(log, f'trainX: {trainX.shape}\t\t trainY: {trainY.shape}')
log_string(log, f'valX:   {valX.shape}\t\tvalY:   {valY.shape}')
log_string(log, f'testX:   {testX.shape}\t\ttestY:   {testY.shape}')
log_string(log, f'mean:   {mean:.4f}\t\tstd:   {std:.4f}')
log_string(log, 'data loaded!')
del trainX, trainTE, valX, valTE, testX, testTE, mean, std
# build model
log_string(log, 'compiling model...')

model = GMAN(SE, args, bn_decay=0.1)
loss_criterion = nn.MSELoss()

optimizer = optim.Adam(model.parameters(), args.learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer,
                                      step_size=args.decay_epoch,
                                      gamma=0.9)
parameters = count_parameters(model)
log_string(log, 'trainable parameters: {:,}'.format(parameters))

time_slot=5, num_his=12, num_pred=12, L=1, K=8, d=8, train_ratio=0.7, val_ratio=0.1, test_ratio=0.2, batch_size=32, max_epoch=1, patience=10, learning_rate=0.001, decay_epoch=10, traffic_file='./data/pems-bay.h5', SE_file='./data/SE(PeMS).txt', model_file='./data/GMAN.pkl', log_file='./data/log'
loading data...
trainX: torch.Size([36458, 12, 325])		 trainY: torch.Size([36458, 12, 325])
valX:   torch.Size([5189, 12, 325])		valY:   torch.Size([5189, 12, 325])
testX:   torch.Size([10400, 12, 325])		testY:   torch.Size([10400, 12, 325])
mean:   62.7354		std:   9.4388
data loaded!
compiling model...
trainable parameters: 209,923


In [18]:
with open('data/GMAN.pkl', 'rb') as f:
    model = pickle.load(f)

UnpicklingError: A load persistent id instruction was encountered,
but no persistent_load function was specified.

In [ ]:
import pandas as pd
import torch

In [ ]:
## THIS IS TO CREATE THE MATRIX PERMUTATION 


# Create a tensor of shape (10, 10)
# Fill the second dimension with values from 1 to 10
tensor = torch.zeros((10, 10))  # Initialize a tensor of zeros

# Assign values from 1 to 10 to the second dimension
for i in range(10):
    tensor[i,:] = torch.arange(1, 11)  # Fill the first column with values 1 to 10

# Print the tensor
print("Tensor:")
print(tensor)

# Given grouping tensor
grouping = torch.tensor([2, 1, 2, 1, 2, 1, 2, 1, 2, 1])

# Initialize a dictionary to keep track of counts for each number
count = {}

# Initialize a new tensor to store the transformed values
transformed = torch.empty_like(grouping)

for index, value in enumerate(grouping):
    # If the value is not in the count dictionary, initialize it
    if value.item() not in count:
        count[value.item()] = 0
    
    # Increment the count for this value
    count[value.item()] += 1
    
    # Calculate the new value using the formula
    k = count[value.item()]  # k is the current count of the number
    new_value = 5*(value.item()-1) + k - 1
    
    # Assign the new value to the transformed tensor
    transformed[index] = new_value

# Print the original and transformed tensors
print("Original Tensor:", grouping)
print("Transformed Tensor:", transformed)

# Create a matrix of zeros with shape (10, 10)
matrix_size = 10
matrix = torch.zeros((matrix_size, matrix_size), dtype=torch.int)

# Iterate through the tensor and set the corresponding positions to 1
for row, value in enumerate(transformed):
    matrix[row, value] = 1  # Set the (row, value + 1) position to 1

# Print the resulting matrix
print("Resulting Matrix:")
print(matrix)

C = torch.matmul(tensor, matrix.float().t())

print(C)

orig = torch.matmul(C, matrix.float())

print(orig)

In [ ]:
## THIS IS FOR READING THE INFO IN PEMS-BAY

import h5py

# Open the HDF5 file
file_path = 'data/pems-bay.h5'  # Replace with your file path

with h5py.File(file_path, 'r') as file:
    # Function to print dataset information
    def print_dataset_info(name, obj):
        print(obj)
        if isinstance(obj, h5py.Dataset):
            print(f"Dataset: {name}")
            print(f"  Type: {obj.dtype}")
            print(f"  Shape: {obj.shape}")
            print(f"  First 5 items: {obj[:5]}\n")  # Print first 5 items, if available

    # Visit all items in the file
    file.visititems(print_dataset_info)
